# Reachability Analysis for Split Computing Neural Networks

## 1. Import Packages

In [1]:
begin
	import Pkg
	Pkg.activate("..")
	Pkg.instantiate()
	
    push!(LOAD_PATH, "$(@__DIR__)/../src")
    push!(LOAD_PATH, "$(@__DIR__)/../Experiments")
    push!(LOAD_PATH, "$(@__DIR__)/../Benchmarks")
    Pkg.add(["Plots", "Distributions", "QuadGK", "ControlSystemsBase", "LinearAlgebra", "ReachabilityAnalysis"])

    
    using Plots
    using Distributions
    using Experiment
    using Models
    using QuadGK
    using ControlSystemsBase
    using LinearAlgebra
    using ReachabilityAnalysis
end

  Activating project at `~/COMP790_CPSEncryption`


   Resolving package versions...


  No Changes to `~/COMP790_CPSEncryption/Project.toml`
  No Changes to `~/COMP790_CPSEncryption/Manifest.toml`


## 2. Validation for Integral function

In [2]:
#const Ts = 0.01
#const Dc = 0.005
#sys = benchmarks[:F1]

In [3]:
#sys_aug_ = let
#    ϕ = ℯ^(Ts * sys.A)
#    Γ₁ = matrix_integral(sys.A, sys.B, Dc, Ts)
#    Γ₀ = matrix_integral(sys.A, sys.B, 0.0, Ts - Dc)
#    ϕ_aug = [ϕ Γ₁; 0 0 0]
#    Γ_aug = [Γ₀; I]
#    C_aug = [sys.C 0]
#    ss(ϕ_aug, Γ_aug, C_aug, sys.D, Ts)
#end

In [4]:
#sys_aug = c2d(sys, Ts) * delay(Dc, Ts)
#sys_aug = c2d(sys, Ts) * thiran(Dc, Ts)

## 3. Physical System and Neural Networks Uncertainty Setup

In [5]:
sys = benchmarks[:F1T]
const period = 0.02
const period_c = 0.001
const Dc =    0.005

0.005

In [6]:
sys_ideal = c2d(sys, period_c)
K_ideal = lqr(ControlSystemsBase.Discrete, sys_ideal.A, sys_ideal.B, I, I)

1×2 Matrix{Float64}:
 0.987398  1.27555

In [7]:
H = 1000
H_c= floor(Int, H * period / period_c)
x0 = 1.
u0 = 0.
n = 1
p = size(sys.A, 1)
q = size(sys.B, 2)
r = size(sys.C, 1)
x0_vec = fill(x0, p)
u_prev = fill(u0, q)
z0 = [x0_vec; u_prev]

3-element Vector{Float64}:
 1.0
 1.0
 0.0

In [8]:
sys_aug = let
    ϕ = ℯ^(period * sys.A)
    Γ₁ = matrix_integral(sys.A, sys.B, Dc, period)
    Γ₀ = matrix_integral(sys.A, sys.B, 0.0, period - Dc)
    ϕ_aug = [ϕ  Γ₁; zeros(q, p)  zeros(q, q)]
    Γ_aug = [Γ₀; I(q)]
    C_aug = [sys.C zeros(size(sys.C,1), q)]
    ss(ϕ_aug, Γ_aug, C_aug, sys.D, period)
end
K = lqr(ControlSystemsBase.Discrete, sys_aug.A, sys_aug.B, I, I)

1×3 Matrix{Float64}:
 0.55255  0.776143  0.221222

## 4. Calculate and Plot Reachable Trajectories

In [22]:
all_trajectories_z = []
all_trajectories_u = []
N = 10
for n in 1:N
    z, u = evolve(sys_aug.A, sys_aug.B, K, H, z0, u0, n)
    push!(all_trajectories_z, z)
    push!(all_trajectories_u, u)
end
z_ideal, u_ideal = ideal_evolve(sys_ideal.A, sys_ideal.B, K_ideal, H_c, x0_vec, u0)

MethodError: MethodError: Cannot `convert` an object of type Vector{Float64} to an object of type Float64

Closest candidates are:
  convert(::Type{R}, !Matched::Espresso.TrackedReal) where R<:Real
   @ Espresso ~/.julia/packages/Espresso/rKtjE/src/tracked.jl:84
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  convert(::Type{T}, !Matched::T) where T
   @ Base Base.jl:84
  ...


In [19]:
const xlim = 2
const ylim = 2
#Plots.scalefontsizes(0.1)

2

In [20]:
traj_plot = plot(
    xlabel = "\$x_1\$",
    ylabel = "\$x_2\$",
    xguidefont = font(24),   # make x-axis label bigger
    yguidefont = font(24),   # make y-axis label bigger
    xtickfont = font(16),    # optional: make tick labels bigger
    ytickfont = font(16)
)
for trajectory in all_trajectories_z
    x_z = [point[1] for point in trajectory]
    y_z = [point[2] for point in trajectory]
    
    plot!(x_z, y_z, xlim=(0, xlim), ylim=(-ylim, ylim), label="", linecolor=:lightgray, linewidth=1)#, marker=:circle, markercolor=:yellow, markersize=2)
end
x_z_ideal = [point[1] for point in z_ideal]
y_z_ideal = [point[2] for point in z_ideal]
plot!(x_z_ideal, y_z_ideal, xlim=(0, xlim+0.2), ylim=(-ylim, ylim), label="", linecolor=:black, linewidth=2)
savefig(traj_plot, "edge_only.png")

"/Users/tingan/COMP790_CPSEncryption/src/edge_only.png"

In [21]:
dev_trajs = [[(z_ideal[1+(k-1)*floor(Int, period / period_c)] .- all_trajectories_z[i][k][1:2]) for k in 1:H] for i in 1:N]
dev₁_trajs = [[dev[1] for dev in dev_traj] for dev_traj in dev_trajs]

max_max_x₁ = maximum(xd -> maximum(xd -> abs(xd[1]), xd), all_trajectories_z)

mean_mean_dev₁ = mean([mean(abs.(dev₁_traj)) for dev₁_traj in dev₁_trajs])
max_max_dev₁ = maximum([maximum(abs.(dev₁_traj)) for dev₁_traj in dev₁_trajs])

std_rmse_dev₁ = std([std(dev₁_traj) for dev₁_traj in dev₁_trajs])
max_rmse_dev₁ = maximum([std(dev₁_traj) for dev₁_traj in dev₁_trajs])
mean_rmse_dev₁ = mean([std(dev₁_traj) for dev₁_traj in dev₁_trajs])

println("Std: ", std_rmse_dev₁)
println("Max: ", max_rmse_dev₁)
println("Mean: ", mean_rmse_dev₁)

Std: 2.4287343061189054e-17
Max: 0.022615617133872477
Mean: 0.0226156171338725
